In [ ]:
import pickle
import sys
import pandas as pd

In [ ]:
data = pickle.load(open('student_answer.pkl','rb'))
hint_data = pickle.load(open('student_hint.pkl','rb'))

attempt = {}
for line in data:
    if line['timestamp'] == 'null':
        continue
    #if line['attempt'] == 'no attempt':
    #    continue
    answer = [line['timestamp'],line['attempt'],line['answer'], line['score']]
    key = (line['set_id'],line['problem_id'],line['part_id'])
    if key not in attempt:
        attempt[key] = {}
    student_key = (str(line['user_id']).replace('L',''),line['username'])
    if student_key not in attempt[key]:
        attempt[key][student_key] = []
    attempt[key][student_key].append(answer)

In [ ]:
### Run this to see the number of wrong attempts for each week each problem and each part.

### Type in integer for the week id
week_id = '5'

attempt_count_data = []
for problem in attempt:
    if problem[0] == week_id:
        total_attempt_number = 0
        for student in attempt[problem]:
            for ans in attempt[problem][student]:
                if ans[3] == '0':
                    total_attempt_number += 1
                elif ans[3] == '1' or ans[3] == '2':
                    break
        attempt_count_data.append([problem[0], problem[1], problem[2], str(total_attempt_number)])

In [ ]:
### Sort the count by attempts
attempt_count_data = sorted(attempt_count_data, key=lambda x: (int(x[1]), int(x[2])))
ind = [a[0] for a in attempt_count_data]
ind2 = [a[1] for a in attempt_count_data]
ind3 = [a[2] for a in attempt_count_data]
d = [a[3] for a in attempt_count_data]
col = ['Total Attempts']
df = pd.DataFrame(d, index = [ind,ind2, ind3], columns = col)
df.index.names = ['Week ID', 'Problem ID', 'Part ID']
df

In [ ]:
### Sort the count by problem id and part id
attempt_count_data = sorted(attempt_count_data, key=lambda x: int(x[3]), reverse = True)
col = ['Week ID', 'Problem ID', 'Part ID', 'Total Attempts']
pd.DataFrame(attempt_count_data, columns = col)

In [ ]:
## Yoav: Generate a single big dataframe for the given week. Generate the smaller reports by selecting from that df.

### Select certain week_id, problem_id, and part_id as int to show below

### Type in integer for the week id
week_id = 5

### If you want to see all problems of certain week, put problem_usr = 0
problem_id = 4

### If you want to see all parts of certain problem,  put part_usr = 0
part_id = 1

In [ ]:
### Run this to output the student attempts and answers.
attempt_data = []
ind0 = []
ind1 = []
ind2 = []
col = ["Attempt", "Answer", "Correctness"]
for problem in attempt:
    if (week_id != 0 and problem[0] != str(week_id)) or \
    (problem_id != 0 and problem[1] != str(problem_id)) or \
    (part_id != 0 and problem[2] != str(part_id)):
        continue 
    print "Data for Week {0}, Problem {1}, Part {2}.".format(problem[0], problem[1], problem[2])
    for student in sorted(attempt[problem],key=lambda x:int(x[0])):
        previous = []
        for ans in attempt[problem][student][:10]:
            if ans[1] == 'null' or ans[1] == '' or ans[2] == 'null':
                continue
            if ans[1:] == previous:
                continue
            ind0.append(student[0])
            ind1.append(student[1])
            ind2.append(ans[0])
            attempt_data.append(ans[1:])
            previous = ans[1:]
ans_df = pd.DataFrame(attempt_data, index = [ind0,ind1,ind2], columns = col)
ans_df.index.names = ['ID', 'Name','TimeStamp']
ans_df

In [ ]:
### Run this to see students' answers with hints.

#combine two dic
def gen_ans_df():
    hint_attempt = {}
    for problem in attempt:
        hint_attempt[problem] = {}
        for student in attempt[problem]:
            if problem in hint_data and student in hint_data[problem]:
                hint_attempt[problem][student] = attempt[problem][student][:]
                for hint_content in hint_data[problem][student]:
                    if hint_content[1] != "no attempt":
                        hint_attempt[problem][student].append(hint_content)
                hint_attempt[problem][student].sort()

    hint_attempt_data = []
    hint_ind0 = []
    hint_ind1 = []
    hint_ind2 = []
    col = ["Attempt", "Attempt/Hint", "Correctness"]
    for problem in hint_attempt:
        if (week_id != 0 and problem[0] != str(week_id)) or \
        (problem_id != 0 and problem[1] != str(problem_id)) or \
        (part_id != 0 and problem[2] != str(part_id)):
            continue 
        print "Data for Week {0}, Problem {1}, Part {2}.".format(problem[0], problem[1], problem[2])
        for student in sorted(hint_attempt[problem], key=lambda x:int(x[0])):
            previous = []
            for ans in hint_attempt[problem][student]:
                if ans[1] == 'null' or ans[1] == '' or ans[2] == 'null':
                    continue
                if ans[1:] == previous:
                    continue
                hint_ind0.append(student[0])
                hint_ind1.append(student[1])
                hint_ind2.append(ans[0])
                hint_attempt_data.append(ans[1:])
                previous = ans[1:]
    ans_df = pd.DataFrame(hint_attempt_data, index = [hint_ind0, hint_ind1, hint_ind2], columns = col)
    ans_df.index.names = ['ID', 'Name','TimeStamp']
    return ans_df

dsdf = gen_ans_df()

In [ ]:
problem=[4,8,3]
ans_df=gen_ans_df()
selector=[ a is None for a in list(ans_df['Correctness'])] 
ans_df[selector]

In [ ]:
print dsdf[:5]